# Part 2: Viewshed case study

In the second part, we will demonstrate the use of GRASS for a small viewshed case study.
The goal is to **compute the area a driver would see from a road**.
This notebook can be run only after notebook Part 1 was executed.

Topics covered:
 * Python scripting
 * manipulating vector data ([v.build.polylines](https://grass.osgeo.org/grass-stable/manuals/v.build.polylines.html), [v.to.points](https://grass.osgeo.org/grass-stable/manuals/v.to.points.html))
 * vector attributes ([v.db.select](https://grass.osgeo.org/grass-stable/manuals/v.db.select.html))
 * viewshed computation ([r.viewshed](https://grass.osgeo.org/grass-stable/manuals/r.viewshed.html))
 * region handling ([grass.script.region_env](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region_env))
 * raster algebra ([r.mapcalc](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html))
 * temporal data handling ([temporal tools](https://grass.osgeo.org/grass-stable/manuals/temporalintro.html))

Load Python libraries. (This may be be skipped in the Google Colab notebook where we already did this  at the beginning of the single, long notebook.)

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS packages we need.
import grass.script as gs
import grass.jupyter as gj

In the previous notebook (Part 1) we created new project *dix_park*. This automatically created new default mapset (subproject) _PERMANENT_ where we then imported our base data. Now it's time to create a new mapset for our viewshed analysis, we will name it _viewshed_:

In [ ]:
from grass.experimental import require_create_ensure_mapset

# Create mapset if it does not exist and continue if it already exists.
require_create_ensure_mapset("dix_park/viewshed", ensure=True)

Schema of GRASS project _dix_park_'s content:

<img src="img/data_structure2.png" alt="GRASS project dix_park" width="400"/>

Start a new session in the mapset:

In [ ]:
# Start GRASS session
session = gj.init("dix_park/viewshed")

## Data preparation
We will first derive viewpoints along the road *Umstead Drive* (vector `umstead_drive_segments`) that we extracted in the first part of the workshop.

Because the road consists of several segments, we will first merge them into one.

In [ ]:
gs.run_command(
    "v.build.polylines",
    input="umstead_drive_segments",
    output="umstead_drive",
    cats="first",
)

Then create new vector of points along the line with distance 50 m:

In [ ]:
gs.run_command(
    "v.to.points", input="umstead_drive", type="line", output="viewpoints", dmax=50
)

Visualize the points with InteractiveMap with OSM tiles (see [other tile options](https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html#module-grass.jupyter.interactivemap)):

In [ ]:
road_map = gj.InteractiveMap(tiles="OpenStreetMap")
road_map.add_vector("umstead_drive")
road_map.add_vector("viewpoints")
road_map.show()

Next part of analysis is raster-based, so we need to make sure we set computational region as we need. Specifically, we set it to match the DSM:

In [ ]:
gs.run_command("g.region", raster="dsm")

Now we want to compute the visibility using DSM, however some points may fall on top of a tree, so we need to filter those out.

First compute height above ground (DSM - DTM) using raster algebra:

In [ ]:
gs.mapcalc("diff = dsm - ground")

Set the color ramp of the raster to "differences", which will highlight in red areas with vegetation and buildings:

In [ ]:
gs.run_command("r.colors", map="diff", color="differences")

diff_map = gj.Map()
diff_map.d_rast(map="diff")
diff_map.d_vect(map="umstead_drive")
diff_map.d_legend(raster="diff")
diff_map.show()

Extract height above ground for the viewpoint locations to identify points that fall on top of a tree growing next to the road:

In [ ]:
gs.run_command("v.what.rast", map="viewpoints", layer=2, raster="diff", column="height")

See the newly computed attribute data. This example shows how the attribute data can be loaded into pandas:

In [ ]:
import json
import pandas as pd

pd.DataFrame(
    json.loads(
        gs.read_command(
            "v.db.select",
            map="viewpoints",
            columns="cat,height",
            layer=2,
            format="json",
        )
    )["records"]
)

Visualize the viewpoints with the height-above-ground raster. You can filter the points based on the height above ground, we won't display points with height > 2.
Additionally, we will render the result larger (`width=1000`) and we will render the map zoomed in to the area with the points
by saving a region and using it in Map (`saved_region="umstead_drive_region"`).

In [ ]:
gs.run_command(
    "g.region",
    vector="umstead_drive",
    align="dsm",
    grow=200,
    save="umstead_drive_region",
)

img = gj.Map(width=1000, saved_region="umstead_drive_region")
img.d_rast(map="diff")
img.d_vect(map="umstead_drive")
img.d_vect(
    map="viewpoints",
    layer=2,
    where="height >= 2",
    size=15,
    icon="basic/pin",
    fill_color="red",
)
img.d_vect(map="viewpoints", layer=2, where="height < 2", size=15, icon="basic/pin")
img.d_legend(raster="diff")
img.show()

## Viewshed computation
We will compute viewsheds from all the viewpoints we generated earlier and from those we compute a cumulative viewshed.
First, we get the list coordinates of the viewpoints that are likely lying on the ground:

In [ ]:
import csv
import io

viewpoints = gs.read_command(
    "v.out.ascii", input="viewpoints", separator="comma", layer=2, where="height < 2"
)
reader = csv.reader(io.StringIO(viewpoints))
viewpoints = list(reader)
viewpoints

We will now compute the viewshed from each viewpoint in a loop. We set max distance of 300 m. Each viewshed will be named `viewshed_{cat}`.

In [ ]:
from tqdm import tqdm

maps = []
for x, y, cat in tqdm(viewpoints):
    cat = int(cat)  # use as a number for formatting
    name = f"viewshed_{cat:02}"  # zero-padding to 2 digits for simple sorting
    gs.run_command(
        "r.viewshed",
        input="dsm",
        output=name,
        coordinates=(x, y),
        max_distance=300,
        flags="b",
    )
    maps.append(name)

In [ ]:
img = gj.Map(width=1000, saved_region="umstead_drive_region")
img.d_rast(map="diff")
for name in tqdm(maps):
    img.d_rast(map=name, values=1)
img.d_vect(map="umstead_drive")
img.d_vect(
    map="viewpoints",
    layer=2,
    where="height >= 2",
    size=15,
    icon="basic/pin",
    fill_color="red",
)
img.d_vect(map="viewpoints", layer=2, where="height < 2", size=15, icon="basic/pin")
img.d_legend(raster="diff")
img.show()

## Temporal dataset of viewsheds

In this part we will create, analyze and visualize a temporal dataset of viewsheds using [temporal tools](https://grass.osgeo.org/grass-stable/manuals/temporal.html). 

First, let's check we have the viewshed rasters ready:

In [ ]:
gs.list_strings(type="raster", pattern="viewshed_*")

We will create an empty space-time raster dataset called _viewshed_ with relative temporal type:

In [ ]:
gs.run_command(
    "t.create",
    output="viewsheds",
    type="strds",
    temporaltype="relative",
    title="Viewshed series",
    description="Series of viewsheds along a road",
)

Now we register the viewshed rasters with start time 1 and 1-minute increment to simulate a change of view of a car driving slowly along the road:

In [ ]:
gs.run_command(
    "t.register",
    input="viewsheds",
    maps=maps,
    start=1,
    unit="minutes",
    increment=1,
)

Let's print basic dataset info. We will use this info later on to set computational region covering the entire dataset.

In [ ]:
info = gs.parse_command("t.info", input="viewsheds", flags="g")
pd.DataFrame(info.values(), index=info.keys())

To list the individual rasters, we will use t.rast.list.

In [ ]:
pd.read_csv(
    io.StringIO(
        gs.read_command(
            "t.rast.list",
            input="viewsheds",
            separator="comma",
            columns="name,start_time",
        )
    )
)

We can quickly get basic statistics such as the size of the viewsheds (see _sum_ column for the number of visible cells):

In [ ]:
df = pd.read_csv(
    io.StringIO(gs.read_command("t.rast.univar", input="viewsheds", separator="comma"))
)
df

Let's find and visualize largest and smallest viewshed:

In [ ]:
largest = df.iloc[df[["sum"]].idxmax()["sum"]].id
smallest = df.iloc[df[["sum"]].idxmin()["sum"]].id

viewshed_map = gj.Map(saved_region="umstead_drive_region")
viewshed_map.d_rast(map="ortho")
viewshed_map.d_rast(map=largest, values=1)
viewshed_map.d_rast(map=smallest, values=1)
viewshed_map.d_vect(map="umstead_drive", color="white")
viewshed_map.show()

Let's compute a temporal dataset where values of each viewshed will represent the registered start time.

We use temporal raster algebra. Here we compute a new temporal dataset _viewsheds_start_ so that for example viewshed with start time 5 has value 5 for visible area and no data for invisible area.

In [ ]:
gs.run_command(
    "t.rast.mapcalc",
    inputs="viewsheds",
    output="viewsheds_start",
    basename="viewshed_start",
    expression="if (viewsheds == 0, null(), start_time())",
)

Set color of the newly computed time series:

In [ ]:
gs.run_command("t.rast.colors", input="viewsheds_start", color="plasma")

With TimeSeriesMap, we can interactively visualize the time series:

In [ ]:
timemap = gj.TimeSeriesMap(width=800)
timemap.d_rast(map="ortho")
timemap.d_vect(map="umstead_drive")
timemap.add_raster_series("viewsheds_start")
timemap.show()

We can export an animated GIF:

In [ ]:
from IPython.display import Image

Image(timemap.save("animation.gif", duration=300))